# Extract cyclone windspeed return period maps

In [ ]:
import os
import multiprocessing
import re
import subprocess
from glob import glob
from pathlib import Path

import matplotlib.pyplot as plt
import pandas
import rasterio

In [ ]:
storm_dir = Path("../../incoming_data/STORM tropical cyclone/")
Path(storm_dir).mkdir(parents=True, exist_ok=True)

out_dir = Path("../../processed_data/hazards/storm_cyclones")
Path(out_dir).mkdir(parents=True, exist_ok=True)

In [ ]:
os.chdir(storm_dir)

In [ ]:
!zenodo_get 7438145

### Tropical cyclone model
To estimate tropical cyclone (TC) wind speed, we use the synthetic cyclone database 
STORM, developed by Bloemendaal et al. (2020) [1]. The STORM database contains 10-meter 
10-minute sustained maximum wind speeds at 10km resolution globally for 26 return periods 
(ranging from 1 year to 10,000 year), estimated for baseline climate conditions and 
for mid-century (2050), RCP8.5.

We create a future time slice, for a mid-century time period, using a scaling 
factor to correct the wind speed based on the expected change in extreme wind. We do this for 
two climate scenarios (RCP4.5 and RCP8.5). In this analysis, we only change the maximum 
wind speed, as these results are robust across models, and do not alter the frequency of certain 
TCs occurring, as there is little consensus on this for the North Atlantic TC basin [2]. 

Based on a review of the studies projecting changes in cyclone wind speed in the North Atlantic basin, 
we identify six relevant studies that use CMIP5 models for their evaluation. 

The end-century ranges provided by these studies are 4-6% increase for RCP4.5 and 6.3-10.5% for 
RCP8.5.

To create a mid-century scenario for RCP4.5, we assume that the increase is approximately linear, 
resulting in a mean increase of 2% under RCP4.5 and 3.5% under RCP8.5.

So to derive a mid-century RCP4.5 scenario from the baseline and mid-century RCP8.5 data,
we interpolate between the baseline and each of the GCM realisations:

        # Extract a change factor for RCP8.5
        wind speed (RCP8.5, gcm) = wind speed (baseline) * change factor (RCP8.5, gcm)

        # Interpolate a change factor for RCP4.5, roughly halfway between baseline and RCP8.5
        change factor (RCP4.5, gcm) = 1 + (0.02/0.035) * (change factor (RCP8.5, gcm) - 1)

        # Calculate a wind speed for RCP4.5
        wind speed (RCP4.5, gcm) = wind speed (baseline) * change factor (RCP4.5, gcm)

1. Bloemendaal, N., Haigh, I.D., de Moel, H., Muis, S., Haarsma, R.J. and Aerts, J.C., 2020. Generation of a global synthetic 
tropical cyclone hazard dataset using STORM. Scientific data, 7(1), pp.1-12.
2. Knutson, T., Camargo, S.J., Chan, J.C., Emanuel, K., Ho, C.H., Kossin, J., Mohapatra, M., Satoh, M., Sugi, M., Walsh, K. 
and Wu, L., 2020. Tropical cyclones and climate change assessment: Part II: Projected response to anthropogenic 
warming. Bulletin of the American Meteorological Society, 101(3), pp. E303-E322.



In [ ]:
fnames = sorted(glob("**/*.tif", recursive=True))
fnames

In [ ]:
countries = {
    "dma": "[-61.804345, 14.941092, -60.884436, 15.811681]",
    "grd": "[-62.131522, 11.760364, -61.262383, 12.678577]",
    "lca": "[-61.426397, 13.465382, -60.506488, 14.335971]",
    "vct": "[-61.637876, 12.652405, -60.768738, 13.570617]",
}

def run_clip(cmd):
    return subprocess.run(cmd, shell=False, capture_output=True)

def generate_commands(fnames, countries, out_dir):
    for fname in fnames:
        matches = re.search(r"STORM_FIXED_RETURN_PERIODS_([^_]+)_(\d+)", fname)
        gcm, rp = matches.groups()

        if "constant" == gcm:
            ssp = "baseline"
            epoch = "2010"
        else:
            ssp = "rcp85"
            epoch = "2050"

        for iso_a3, extent in countries.items():
            out_fname  = out_dir / f"cyclone_windspeed__epoch_{epoch}__rcp_{ssp}__gcm_{gcm}__rp_{rp}__isoa3_{iso_a3}.tif"
            # Run: rio clip --bounds-[xmin,ymin,xmax,ymax] input.tif output.tif
            cmd = [
                "rio",
                "clip",
                f"--bounds={extent}",
                str(fname),
                str(out_fname),
            ]
            yield cmd

count = multiprocessing.cpu_count() - 2
pool = multiprocessing.Pool(processes=count)
results = pool.map(run_clip, generate_commands(fnames, countries, out_dir))
len(results)

In [ ]:
results[0].stdout, results[0].stderr

In [ ]:
base_dir = Path("../..")
os.chdir(base_dir)

In [ ]:

base_out_dir = Path("./processed_data/hazards/storm_cyclones")
out_fnames = sorted(glob(str(base_out_dir / "*.tif")))
windspeed_files = pandas.DataFrame({'fname': out_fnames})
extract_cols = ["hazard", "epoch", "rcp", "gcm", "rp", "isoa3"]
extract_pattern = r"(\w+)__epoch_(\d+)__rcp_(\w+)__gcm_([^_]+)__rp_(\w+)__isoa3_(\w+)"
windspeed_files["key"] = windspeed_files.fname.apply(lambda f: Path(f).stem)
windspeed_files[extract_cols] = windspeed_files.key.str.extract(extract_pattern)
windspeed_files

In [ ]:
def get_paths(row):
    """Return future (RCP8.5) and equivalent rp/country/baseline paths for a hazard map
    """
    epoch = row.epoch
    ssp = row.rcp
    gcm = row.gcm
    rp = row.rp
    iso_a3 = row.isoa3
    return (
        base_out_dir / f"cyclone_windspeed__epoch_{epoch}__rcp_{ssp}__gcm_{gcm}__rp_{rp}__isoa3_{iso_a3}.tif",
        base_out_dir / f"cyclone_windspeed__epoch_{epoch}__rcp_rcp45__gcm_{gcm}__rp_{rp}__isoa3_{iso_a3}.tif",
        base_out_dir / f"cyclone_windspeed__epoch_2010__rcp_baseline__gcm_constant__rp_{rp}__isoa3_{iso_a3}.tif"
    )

In [ ]:
def read_ws(fname_baseline, fname_rcp85):
    with rasterio.open(fname_rcp85) as src:
        ws_rcp85 = src.read(1)

    with rasterio.open(fname_baseline) as src:
        ws_baseline = src.read(1)

    return ws_baseline, ws_rcp85, (src.width, src.height, src.crs, src.transform)

In [ ]:
def interpolate_ws_rcp45(ws_baseline, ws_rcp85):
    # Extract a change factor for RCP8.5
    f_rcp85 = ws_rcp85 / ws_baseline

    # Interpolate a change factor for RCP4.5, roughly halfway between baseline and RCP8.5
    f_rcp45 = 1 + (0.02/0.035) * (f_rcp85 - 1)

    # Calculate a wind speed for RCP4.5
    ws_rcp45 = ws_baseline * f_rcp45

    return ws_rcp45

In [ ]:
row = windspeed_files[windspeed_files.rcp == "rcp85"].iloc[15]
row

In [ ]:
fname_rcp85, fname_rcp45, fname_baseline = get_paths(row)
ws_baseline, ws_rcp85, (width, height, crs, transform) = read_ws(fname_baseline, fname_rcp85)
ws_rcp45 = interpolate_ws_rcp45(ws_baseline, ws_rcp85)
vmin, vmax = ws_baseline.min(), ws_rcp85.max()
fig, axs = plt.subplots(nrows=1, ncols=3)
axs[0].imshow(ws_baseline, vmin=vmin, vmax=vmax)
axs[1].imshow(ws_rcp45, vmin=vmin, vmax=vmax)
axs[2].imshow(ws_rcp85, vmin=vmin, vmax=vmax)


In [ ]:
future_files = windspeed_files[windspeed_files.rcp == "rcp85"].copy()
future_files.rcp.value_counts()

In [ ]:
for row in future_files.itertuples():
    fname_rcp85, fname_rcp45, fname_baseline = get_paths(row)
    ws_baseline, ws_rcp85, (width, height, crs, transform) = read_ws(fname_baseline, fname_rcp85)
    ws_rcp45 = interpolate_ws_rcp45(ws_baseline, ws_rcp85)
    with rasterio.open(
            fname_rcp45,
            'w',
            driver='GTiff',
            height=height,
            width=width,
            count=1,
            dtype=ws_rcp45.dtype,
            crs=crs,
            transform=transform,
        ) as dst:
        dst.write(ws_rcp45, 1)